# Snapshots
## Using Snapshots to enable business continuty for your data

Snapshots are zero space copies of what the data on your volume looked like for a particluar point in time.

<p align="center">
  <img src="Pictures/edf-snapshots-overview.png" width="600" height="400" alt="Data Fabric Snapshots">                                                    
  
</p>

### Features of Ezmeral Data Fabric Snapshots:

- Consistent, Point-In-Time image of a volume
- Can be created on-demand or scheduled
- Lightning fast creation, even at 1PB of data
- Stores incremental changes
- Uses Redirect on-write technology, which saves space on changes
- Zero performance loss on writing to original data
- Proven and battle-tested technology in the storage industry


In [1]:
%login 16.31.86.198

echo "We are now connected to the appliance"

#Variables used by scripts
#password="qcpx0k4u" # your Data Fabric login credentials - password - 
#apiserver_host="16.31.85.156" # API Server for the Data Fabric Cluster
#ad_group="data-fabric-users" # LDAP Domain Group for Data Fabric users
#vol_name="vol1" # suffix for student volume name
#snapvol_name="snapvol1" # suffix for student snap volume name
#cluster_name="mapr-cluster2.etc.fr.comm.hpecorp.net" # Data Fabric Cluster Name
#src_mirrvol1="src-mirrvol1" # suffix for student source mirror volume name
#dst_mirrvol1="dst-mirrvol1"  # suffix for student destination mirror volume name
#secvol_name="sec-vol1"  # suffix for student volume name used for security comparison / user access 
#sec_policy="student1222-policy" #security policy to allow r/w access to volumes

echo "your operation context is:" student1222 "with Password:" qcpx0k4u

[ssh] Login to 16.31.86.198...
[ssh] host=16.31.86.198 hostname=16.31.86.198 other_conf={'user': None, 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f18f4615520>}
[ssh] Successfully logged in.
[ssh] host = 16.31.86.198, cwd = /student/student1222
We are now connected to the appliance
your operation context is: student1222 with Password: qcpx0k4u


## 1. Creating a snapshot

#### Background:

You have been given a user that is allowed to create and modify volumes on the cluster.  Data Fabric volumes can be mounted so they are accessable from NFS, HDFS or POSIX commands (like the bash kernel shell in these labs).  For security reasons, you can only mount volumes in the /user directory.

First, let's create a volume to work with and create snapshots on:

In [2]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/create?name='$username-$snapvol_name'&path=/user/'$username-$snapvol_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   178  100   178    0     0     70      0  0:00:02  0:00:02 --:--:--    70
{"timestamp":1624511511010,"timeofday":"2021-06-24 07:11:51.010 GMT+0200 AM","status":"OK","total":0,"data":[],"messages":["Successfully created volume: 'student1222-snapvol1'"]}

In order to use and demonstrate snapshots effectively we need to write some data to our volume and then verify the data was created.

In [3]:
source ~/.bashrc
echo "## Creating a file before we take a snapshot.##"
echo "$username-$snapvol_name snapshot volume update $(date)" >> /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1
echo "## Listing out our directory to view the file we just created.##"
ls /mapr/$cluster_name/user/$username-$snapvol_name
echo "## Verify the file we just wrote has the correct contents.##"
cat /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1

[ssh] host = 16.31.86.198, cwd = /student/student1222
## Creating a file before we take a snapshot.##
## Listing out our directory to view the file we just created.##
student1222-pre-snapshot-file1
## Verify the file we just wrote has the correct contents.##
student1222-snapvol1 snapshot volume update Thu Jun 24 07:11:15 CEST 2021


Now that we have created the volume and created data within it we can create an "On-demand" snapshot:

In [4]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/create?volume='$username-$snapvol_name'&snapshotname='$username-snapshot1'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0     0    288      0 --:--:-- --:--:-- --:--:--   289
{"timestamp":1624511511682,"timeofday":"2021-06-24 07:11:51.682 GMT+0200 AM","status":"OK","total":0,"data":[]}

Lastly, let's create another file to showcase the difference in file listing of the snapshot in a future step.

In [5]:
source ~/.bashrc
echo "## Creating a file after we took a snapshot.##"
echo "$username-$snapvol_name snapshot volume update $(date)" >> /mapr/$cluster_name/user/$username-$snapvol_name/$username-post-snapshot-file1
echo "## Listing out our directory to view the file we just created.##"
ls /mapr/$cluster_name/user/$username-$snapvol_name
echo "## Verify the file we just wrote has the correct contents.##"
cat /mapr/$cluster_name/user/$username-$snapvol_name/$username-post-snapshot-file1

[ssh] host = 16.31.86.198, cwd = /student/student1222
## Creating a file after we took a snapshot.##
## Listing out our directory to view the file we just created.##
student1222-post-snapshot-file1
student1222-pre-snapshot-file1
## Verify the file we just wrote has the correct contents.##
student1222-snapvol1 snapshot volume update Thu Jun 24 07:11:23 CEST 2021


## 2. Listing the snapshot you just created:

In [6]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/list' --user $username:$password | jq -r | grep -B 12 -A 2 ''$username-$snapvol_name''

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   622  100   622    0     0   2005      0 --:--:-- --:--:-- --:--:--  2012
{
  "timestamp": 1624511519806,
  "timeofday": "2021-06-24 07:11:59.806 GMT+0200 AM",
  "status": "OK",
  "total": 1,
  "data": [
    {
      "ownername": "student1222",
      "ownertype": "1",
      "volumeid": "230988410",
      "volumename": "student1222-snapvol1",
      "volumepath": "/user/student1222-snapvol1",
      "snapshotid": "256000073",
      "snapshotname": "student1222-snapshot1",


## 3. Viewing data within a snapshot

By default all Snapshots, when created, can be viewed by looking within an explicit hidden directory at the top level within the volume.
For example if you try and list the contents of the top level directory you will not see the hidden dirctory

Example:

In [7]:
source ~/.bashrc

ls -la /mapr/$cluster_name/user/$username-$snapvol_name/

[ssh] host = 16.31.86.198, cwd = /student/student1222
total 2
drwxr-xr-x. 2 student1222 student1222        2 Jun 24  2021 .
drwxrwxrwx. 5        5000 Data-Fabric-Users  3 Jun 24  2021 ..
-rw-r--r--. 1 student1222 student1222       74 Jun 24  2021 student1222-post-snapshot-file1
-rw-r--r--. 1 student1222 student1222       74 Jun 24  2021 student1222-pre-snapshot-file1


But if we explicitly set the '.snapshot' directory in our listing we will see our snapshot.

Example:

In [8]:
source ~/.bashrc

ls /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot

[ssh] host = 16.31.86.198, cwd = /student/student1222
student1222-snapshot1


The above listing is your manual snapshot you created in step 1.  Now let's look at the contents within this .snapshot directory

Example:


In [9]:
source ~/.bashrc

ls /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot/$username-snapshot1

[ssh] host = 16.31.86.198, cwd = /student/student1222
student1222-pre-snapshot-file1


In [10]:
source ~/.bashrc

echo "If this section was successful you should see your test file you created called:"
echo "$username-pre-snapshot-file1"

[ssh] host = 16.31.86.198, cwd = /student/student1222
If this section was successful you should see your test file you created called:
student1222-pre-snapshot-file1


## 4. Recovery specific data within a snapshot

Now let's delete some data!


In [11]:
source ~/.bashrc

rm /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1

[ssh] host = 16.31.86.198, cwd = /student/student1222


Verify the previous test files we created is removed:

In [12]:
source ~/.bashrc

ls /mapr/$cluster_name/user/$username-$snapvol_name/

[ssh] host = 16.31.86.198, cwd = /student/student1222
student1222-post-snapshot-file1


Now let's recovery the specific file from your snapshot, first let's take a look at data within your snapshot.

In [13]:
source ~/.bashrc

ls /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot/$username-snapshot1

[ssh] host = 16.31.86.198, cwd = /student/student1222
student1222-pre-snapshot-file1


Verify it's the file you want:

In [14]:
source ~/.bashrc

cat /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot/$username-snapshot1/$username-pre-snapshot-file1

[ssh] host = 16.31.86.198, cwd = /student/student1222
student1222-snapvol1 snapshot volume update Thu Jun 24 07:11:15 CEST 2021


Recover the individual file from your snapshot to your  volume:

In [15]:
source ~/.bashrc

cp /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot/$username-snapshot1/$username-pre-snapshot-file1 /mapr/$cluster_name/user/$username-$snapvol_name/

[ssh] host = 16.31.86.198, cwd = /student/student1222


Verify your file is on your volume:

In [16]:
source ~/.bashrc
echo "##List out the files in your volume.##"
ls /mapr/$cluster_name/user/$username-$snapvol_name/
echo "##Verify the contents of your file you recovered.##"
cat /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1

[ssh] host = 16.31.86.198, cwd = /student/student1222
##List out the files in your volume.##
student1222-post-snapshot-file1
student1222-pre-snapshot-file1
##Verify the contents of your file you recovered.##
student1222-snapvol1 snapshot volume update Thu Jun 24 07:11:15 CEST 2021


## 5. Roll back your data in a volume using snapshot restore

#### Another feature is the abilty to roll your volumes data back to a point-in-time snapshot with Snapshot Restore

With Snapshot Restore you need:

- A currently created volume.
- A volume with a snapshot.
- You must unmount and then mount the volume again

Snapshot Restore is a disributive process and should only be used when you are declaring a diaster or a business continutity event on your data.

First let's unmount our volume:

In [17]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/unmount?name='$username-$snapvol_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0     0     47      0  0:00:02  0:00:02 --:--:--    47
{"timestamp":1624511541969,"timeofday":"2021-06-24 07:12:21.969 GMT+0200 AM","status":"OK","total":0,"data":[]}

Now you can restore your snapshot to a scheudled or on-demand snapshot

In [18]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/restore?snapshotname='$username-snapshot1'&volume='$username-$snapvol_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   219  100   219    0     0    792      0 --:--:-- --:--:-- --:--:--   793
{"timestamp":1624511542333,"timeofday":"2021-06-24 07:12:22.333 GMT+0200 AM","status":"OK","total":0,"data":[],"messages":["Snapshot Restore queued for volume student1222-snapvol1, snapshot name student1222-snapshot1"]}

And then re-mount your volume

In [19]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/mount?name='$username-$snapvol_name'&path=/user/'$username-$snapvol_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   214  100   214    0     0    512      0 --:--:-- --:--:-- --:--:--   513
{"timestamp":1624511542836,"timeofday":"2021-06-24 07:12:22.836 GMT+0200 AM","status":"ERROR","errors":[{"id":10019,"desc":"Failed to mount volume student1222-snapvol1, snapshot restore operation is in progress"}]}

Now, verify your files are restored to your snapshot restore time:

In [20]:
source ~/.bashrc
echo "##List out the files in your volume.##"
ls /mapr/$cluster_name/user/$username-$snapvol_name/
echo "##Verify the contents of your file you recovered.##"
cat /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1

[ssh] host = 16.31.86.198, cwd = /student/student1222
##List out the files in your volume.##
ls: cannot access /mapr/mapr-cluster2.etc.fr.comm.hpecorp.net/user/student1222-snapvol1/: No such file or directory
##Verify the contents of your file you recovered.##
cat: /mapr/mapr-cluster2.etc.fr.comm.hpecorp.net/user/student1222-snapvol1/student1222-pre-snapshot-file1: No such file or directory


abnormal exit code: 1

For more information on the Snapshot restore feature please see [Ezmeral Data Fabric: Snapshot Restore](https://docs.datafabric.hpe.com/62/AdministratorGuide/Snapshots-Restoring-new.html?hl=snapshot%2Crestore)

## 6. Deleting Snapshots 
### Let's cleanup now

 1. Remove volumes
 2. Remove snapshots

In [21]:
source ~/.bashrc
echo "Deleting Lab2 snapshots"
echo "Listing Student snapshot and extracting right one with associated snapshot id"
snapid=$(curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/list'| jq -r | grep -B 12 -A 2 ''$username-$snapvol_name'' | grep snapshotid | cut -c22-30)

echo "Deleting LAB2 Snapshot using its id"
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/remove?snapshots='$snapid

echo "Deleting LAB2 student Snapvol volume"
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/remove?name='$username-$snapvol_name'' --user $username:$password


[ssh] host = 16.31.86.198, cwd = /student/student1222
Deleting Lab2 snapshots
Listing Student snapshot and extracting right one with associated snapshot id
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   582  100   582    0     0   2055      0 --:--:-- --:--:-- --:--:--  2063
Deleting LAB2 Snapshot using its id
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0     0    402      0 --:--:-- --:--:-- --:--:--   402
{"timestamp":1624511562064,"timeofday":"2021-06-24 07:12:42.064 GMT+0200 AM","status":"OK","total":0,"data":[]}Deleting LAB2 student Snapvol volume
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0   

In [22]:
%logout 

[ssh] Closing existing connection.
[ssh] Successfully logged out.


## Snapshots Summary

In this example we:

1. Created a volume
2. Placed data in that volume
3. Created a snapshot and then showed how to recover individual data from a snapshot back to into the volume
4. We also showed how to "Roll Back" your entire volumes to a particular snapshot point in time by using Snapshot Restore.

# Using Mirrror's for Disaster Recovery
Let's now showcase how to use the Mirror feature on a volume for disater recovery or for developer tasks.

* [Lab 3 - Using Mirror's for Disaster Recovery](3-WKSHP-Mirrors.ipynb)